********
# Neo
********

This figure shows the main data types in Neo, with the exception of the newly added ImageSequence and RegionOfInterest classes:

<img src='images/base_schematic.png' width="400" height="400">

Neo objects fall into three categories: data objects, container objects and grouping objects.

### Data objects
***********
These objects directly represent data as arrays of numerical values with
associated metadata (units, sampling frequency, etc.).

  * :py:class:`AnalogSignal`: A regular sampling of a single- or multi-channel continuous analog signal.
  * :py:class:`SpikeTrain`: A set of action potentials (spikes) emitted by the same unit in a period of time (with optional waveforms).

### Container objects
*************

There is a simple hierarchy of containers:

  * :py:class:`Segment`: A container for heterogeneous discrete or continuous data sharing a common
    clock (time basis) but not necessarily the same sampling rate, start time or end time.
    A :py:class:`Segment` can be considered as equivalent to a "trial", "episode", "run",
    "recording", etc., depending on the experimental context.
    May contain any of the data objects.
  * :py:class:`Block`: The top-level container gathering all of the data, discrete and continuous,
    for a given recording session.
    Contains :class:`Segment` and :class:`Group` objects.

### Grouping/linking objects
------------------------

* :py:class:`Group`: Can contain any of the data objects, views, or other groups,
    outside the hierarchy of the segment and block containers.
    A common use is to link the :class:`SpikeTrain` objects within a :class:`Block`,
    possibly across multiple Segments, that were emitted by the same neuron.
    
<img src='images/multi_segment_diagram_spiketrain.png' width="400" height="400">

### NumPy compatibility
===================

Neo data objects inherit from :py:class:`Quantity`, which in turn inherits from NumPy
:py:class:`ndarray`. This means that a Neo :py:class:`AnalogSignal` is also a :py:class:`Quantity`
and an array, giving you access to all of the methods available for those objects.

For example, you can pass a :py:class:`SpikeTrain` directly to the :py:func:`numpy.histogram`
function, or an :py:class:`AnalogSignal` directly to the :py:func:`numpy.std` function.

If you want to get a numpy.ndarray you use magnitude and rescale from quantities::

In [ ]:
np_sig = neo_analogsignal.rescale('mV').magnitude
np_times = neo_analogsignal.times.rescale('s').magnitude

### Relationships between objects
=============================

Container objects like :py:class:`Block` or :py:class:`Segment` are gateways to
access other objects. For example, a :class:`Block` can access a :class:`Segment`
with::

    >>> bl = Block()
    >>> bl.segments
    # gives a list of segments

A :class:`Segment` can access the :class:`AnalogSignal` objects that it contains with::

    >>> seg = Segment()
    >>> seg.analogsignals
    # gives a list of AnalogSignals

In the :ref:`neo_diagram` below, these *one to many* relationships are represented by cyan arrows.
In general, an object can access its children with an attribute *childname+s* in lower case, e.g.

    * :attr:`Block.segments`
    * :attr:`Segments.analogsignals`
    * :attr:`Segments.spiketrains`
    * :attr:`Block.groups`

These relationships are bi-directional, i.e. a child object can access its parent:

    * :attr:`Segment.block`
    * :attr:`AnalogSignal.segment`
    * :attr:`SpikeTrain.segment`
    * :attr:`Group.block`

Here is an example showing these relationships in use::

In [ ]:
# example here

### Neo diagram
===========

Object:
  * With a star = inherits from :class:`Quantity`
Attributes:
  * In red = required
  * In white = recommended
Relationship:
  * In cyan = one to many
  * In yellow = properties (deduced from other relationships)


<img src='images/simple_generated_diagram.png' width="700" height="400">

### Initialization
==============

Neo objects are initialized with "required", "recommended", and "additional" arguments.

    - Required arguments MUST be provided at the time of initialization. They are used in the construction of the object.
    - Recommended arguments may be provided at the time of initialization. They are accessible as Python attributes. They can also be set or modified after initialization.
    - Additional arguments are defined by the user and are not part of the Neo object model. A primary goal of the Neo project is extensibility. These additional arguments are entries in an attribute of the object: a Python dict called :py:attr:`annotations`.
      Note : Neo annotations are not the same as the *__annotations__* attribute introduced in Python 3.6.

### Example: SpikeTrain
-------------------

:py:class:`SpikeTrain` is a :py:class:`Quantity`, which is a NumPy array containing values with physical dimensions. The spike times are a required attribute, because the dimensionality of the spike times determines the way in which the :py:class:`Quantity` is constructed.

Here is how you initialize a :py:class:`SpikeTrain` with required arguments::

In [ ]:
import neo
st = neo.SpikeTrain([3, 4, 5], units='sec', t_stop=10.0)
st

You will see the spike times printed in a nice format including the units.
Because `st` "is a" :py:class:`Quantity` array with units of seconds, it absolutely must have this information at the time of initialization. You can specify the spike times with a keyword argument too::

In [ ]:
st = neo.SpikeTrain(times=[3, 4, 5], units='sec', t_stop=10.0)

The spike times could also be in a NumPy array.

If it is not specified, :attr:`t_start` is assumed to be zero, but another value can easily be specified::

In [ ]:
st = neo.SpikeTrain(times=[3, 4, 5], units='sec', t_start=1.0, t_stop=10.0)
st.t_start

Recommended attributes must be specified as keyword arguments, not positional arguments.


Finally, let's consider "additional arguments". These are the ones you define for your experiment::

In [ ]:
st = neo.SpikeTrain(times=[3, 4, 5], units='sec', t_stop=10.0, rat_name='Fred')
st.annotations

Because ``rat_name`` is not part of the Neo object model, it is placed in the dict :py:attr:`annotations`. This dict can be modified as necessary by your code.

### Annotations
-----------

As well as adding annotations as "additional" arguments when an object is
constructed, objects may be annotated using the :meth:`annotate` method
possessed by all Neo core objects, e.g.::

In [ ]:
seg = Segment()
seg.annotate(stimulus="step pulse", amplitude=10*nA)
seg.annotations

Since annotations may be written to a file or database, there are some
limitations on the data types of annotations: they must be "simple" types or
containers (lists, dicts, tuples, NumPy arrays) of simple types, where the simple types
are ``integer``, ``float``, ``complex``, ``Quantity``, ``string``, ``date``, ``time`` and
``datetime``.

### Array Annotations
-----------------

Next to "regular" annotations there is also a way to annotate arrays of values
in order to create annotations with one value per data point. Using this feature,
called Array Annotations, the consistency of those annotations with the actual data
is ensured.
Apart from adding those on object construction, Array Annotations can also be added
using the :meth:`array_annotate` method provided by all Neo data objects, e.g.::

In [ ]:
sptr = SpikeTrain(times=[1, 2, 3]*pq.s, t_stop=3*pq.s)
sptr.array_annotate(index=[0, 1, 2], relevant=[True, False, True])
sptr.array_annotations

Since Array Annotations may be written to a file or database, there are some
limitations on the data types of arrays: they must be 1-dimensional (i.e. not nested)
and contain the same types as annotations:

``integer``, ``float``, ``complex``, ``Quantity``, ``string``, ``date``, ``time`` and ``datetime``.